In [11]:
!py -m pip install pyodbc pandas

In [12]:
import pyodbc
import pandas as pd
import json

In [13]:
# Open and read the JSON file
with open('config/mom_db_config.json', 'r') as file:
    db_config = json.load(file)

# Print the data
print(db_config)

{'driver': 'DRIVER={ODBC Driver 17 for SQL Server};', 'server': 'SERVER=V19XX002;', 'database': 'DATABASE=MOM-PC;', 'user': 'UID=Danny.Sloan;', 'password': 'PWD=1$d*Y|0ztA42;'}


In [14]:
# Example for SQL Server Authentication
conn_str = (
    f"{db_config['driver']}"
    f"{db_config['server']}"
    f"{db_config['database']}"
    f"{db_config['user']}"
    f"{db_config['password']}"
)

print(conn_str)
# Create connection
conn = pyodbc.connect(conn_str)

DRIVER={ODBC Driver 17 for SQL Server};SERVER=V19XX002;DATABASE=MOM-PC;UID=Danny.Sloan;PWD=1$d*Y|0ztA42;


In [15]:
query = "SELECT TOP 10 * FROM CMS"
df = pd.read_sql(query, conn)

C:\Users\danny.sloan\AppData\Local\Temp\ipykernel_34408\308473102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [16]:
df.head()

,ORDERNO,CUSTNUM,CL_KEY,ODR_DATE,HOLD_TYPE,PERM_HOLD,SYS_HOLD,HOLD_DATE,REL_DATE,CLEAR_DATE,...,PICKED_UP,PICKUPWARE,TAXSERVICE,TRANFEE,TB_TRANFEE,TFALLOWED,TFMODIFY,FBA,AVALARA,AV_ENTITY
0,1.0,42850.0,PC100,2004-03-31,,False,False,None,None,None,...,False,,False,0.0,0.0,False,False,False,False,
1,2.0,42850.0,PC730,2004-03-31,,False,False,None,None,None,...,False,,False,0.0,0.0,False,False,False,False,
2,3.0,42852.0,10T07,2004-03-31,,False,False,None,None,None,...,False,,False,0.0,0.0,False,False,False,False,
3,4.0,42850.0,,2004-03-31,,False,False,None,None,None,...,False,,False,0.0,0.0,False,False,False,False,
4,5.0,38652.0,11T08,2004-03-31,,False,False,None,None,None,...,False,,False,0.0,0.0,False,False,False,False,


In [17]:
conn.close()